In [3]:
from Acesso import Login
from Query import Query
import pandas as pd

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

querys={

    'TargetEstatico':

    """
    
    DECLARE @DTBASE DATETIME,@DTFIM DATETIME,@DTINICIO DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DTFIM=@DTBASE

    SET @DTINICIO=CONCAT(YEAR(@DTFIM),'-',MONTH(@DTFIM),'-01')

    SELECT * FROM netfeira.vw_targetestatico
    WHERE [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM
    ORDER BY [Data de Faturamento]

    
    """,
    
    'Aberto':
    
    """
    
    SELECT * FROM netfeira.vw_aberto
    WHERE [Data do Pedido]=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)
    
    """,
    
    'Meta':
    
    """
    
    SELECT * FROM netfeira.vw_metas
    
    """,
    
    'Calendario':
    
    
    """
    
    SELECT * FROM netfeira.vw_calend
    WHERE YEAR(Data)=YEAR(GETDATE()) AND MONTH(Data)=MONTH(GETDATE())
    
    """,
    
    'Vendedor':
    
    """
    
    SELECT * FROM netfeira.vw_vendedor
    
    """,
    
    
    'Supervisor':
    
    
    """
    
    SELECT * FROM netfeira.vw_supervisor
    
    """

}

In [4]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [5]:
vendas_df=pd.DataFrame()

vendas_df=tabelas_df['TargetEstatico'].loc[tabelas_df['TargetEstatico']['Tipo de Operação']=='VENDAS']

In [6]:
base_df=pd.DataFrame()

base_df=vendas_df[['Situação','Total Venda']].groupby(['Situação'],as_index=False).sum()

In [7]:
base_df['Pedido']=base_df['Situação'].apply(lambda info: len(vendas_df['Pedido'].loc[vendas_df['Situação']==info].unique().tolist()))

base_df['Cliente']=base_df['Situação'].apply(lambda info: len(vendas_df['ID Cliente'].loc[vendas_df['Situação']==info].unique().tolist()))

base_df['MIX']=base_df['Situação'].apply(lambda info: len(vendas_df['SKU'].loc[vendas_df['Situação']==info].unique().tolist()))

In [8]:
base_df.loc[base_df['Situação']!='FATURADO','Pedido']=base_df.loc[base_df['Situação']!='FATURADO','Pedido']*-1

base_df.loc[base_df['Situação']!='FATURADO','Cliente']=base_df.loc[base_df['Situação']!='FATURADO','Cliente']*-1

base_df.loc[base_df['Situação']!='FATURADO','MIX']=base_df.loc[base_df['Situação']!='FATURADO','MIX']*-1

In [15]:
pedido=base_df['Pedido'].sum()

cliente=base_df['Cliente'].sum()

faturado=round(base_df['Total Venda'].sum(),2)

meta=round(tabelas_df['Meta']['Meta R$'].sum(),2)

perc_meta=round(faturado/meta,4)*100 if meta>0 else 0

dif_meta=faturado-meta

ticket=round(faturado/pedido,2)

In [16]:
calend_df=pd.DataFrame()

calend_df=tabelas_df['Calendario']

uteis=len(calend_df['Data'].loc[calend_df['Dia Útil']==True].unique().tolist())

trabalhado=len(calend_df['Data Trabalhada'].loc[(calend_df['Dia Útil']==True)&(~calend_df['Data Trabalhada'].isnull())].unique().tolist())-1

restante=uteis-trabalhado

In [31]:
projecao=(faturado/trabalhado)*uteis if trabalhado>0 else 0

meta_diaria=round(meta/uteis,2)

0

In [32]:
aberto_df=pd.DataFrame()

aberto_df=tabelas_df['Aberto']

atendimento=len(aberto_df['ID Cliente'].unique().tolist())

realizado=round(aberto_df['Total Geral'].sum(),2)

ped_realizado=len(aberto_df['Pedido'].unique().tolist())

perc_diario=round(realizado/meta_diaria,4)*100 if meta_diaria>0 else 0

real_aberto=round(aberto_df['Total Geral'].loc[aberto_df['Situação']=='AB'].sum(),2)

total=faturado+real_aberto

kg_real=round(aberto_df['Peso Bruto KG'].sum(),3)

mix=len(aberto_df['SKU'].unique().tolist())

dif_diario=round(total-meta_diaria,2)

In [33]:
tabelas_df['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro', 'Status do Vendedor'],
      dtype='object')

In [34]:
tabelas_df['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente'],
      dtype='object')

In [35]:
equipes_df=pd.DataFrame()

equipes_df=tabelas_df['Vendedor'].merge(tabelas_df['Supervisor'],on='ID Equipe',how='inner')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe','Supervisor']]

In [36]:
equipes_df=equipes_df.merge(tabelas_df['Meta'],on='ID Vendedor',how='inner')

In [37]:
equipes_df=equipes_df.loc[equipes_df['Meta R$']>0]

In [38]:
equipes_df.loc[:,'Diário']=round(equipes_df['Meta R$']/uteis,2)

In [39]:
equipes_df

,ID Vendedor,Vendedor,Nome Resumido,Equipe,Supervisor,Meta R$,Diário


In [40]:
aberto_df.columns

Index(['ID Empresa', 'Pedido', 'ID Cliente', 'ID Vendedor', 'Data do Pedido',
       'SKU', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA', 'Total Venda',
       'Total AV', 'Total Geral', 'Margem Bruta R$', 'Margem Bruta CRP',
       'Situação', 'Peso Bruto KG', 'Peso Líquido KG'],
      dtype='object')

In [41]:
vendedores_df=pd.DataFrame()

vendedores_df=aberto_df[['ID Vendedor','Total Venda']].groupby(['ID Vendedor'],as_index=False).sum()

In [42]:
equipes_df=equipes_df.merge(vendedores_df,on='ID Vendedor',how='left')

In [43]:
equipes_df.loc[equipes_df['Total Venda'].isnull(),'Total Venda']=0

In [49]:
equipes_df.loc['Diferença']=0

if(len(equipes_df)>0):

    equipes_df['Diferença']=equipes_df.apply(lambda info: info['Total Venda']-info['Diário'],axis=1)
    
    pass

In [50]:
consolidado_df=pd.DataFrame()

consolidado_df=equipes_df[['Supervisor','Diferença']].groupby(['Supervisor'],as_index=False).sum()

In [51]:
consolidado_df['Diferença']=consolidado_df['Diferença']*-1

In [52]:
consolidado_df.sort_values('Diferença',ascending=False,inplace=True)

In [53]:
consolidado_df=consolidado_df.loc[consolidado_df['Diferença']>0]

In [54]:
consolidado_df=equipes_df[['Equipe','Total Venda','Diferença']].groupby(['Equipe'],as_index=False).sum()

consolidado_df.rename(columns={'Total Venda':'Realizado R$'},inplace=True)

consolidado_df.sort_values('Realizado R$',ascending=False,inplace=True)

consolidado_df=consolidado_df[['Equipe','Realizado R$']].loc[consolidado_df['Diferença']>0]

In [55]:
aberto_df.columns

Index(['ID Empresa', 'Pedido', 'ID Cliente', 'ID Vendedor', 'Data do Pedido',
       'SKU', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA', 'Total Venda',
       'Total AV', 'Total Geral', 'Margem Bruta R$', 'Margem Bruta CRP',
       'Situação', 'Peso Bruto KG', 'Peso Líquido KG'],
      dtype='object')

In [56]:
vendas_df=vendas_df.merge(tabelas_df['Vendedor'],on='ID Vendedor',how='inner')[['ID Empresa', 'Pedido', 'Nfe', 'ID Cliente', 'ID Vendedor',
       'Data de Emissão', 'Data de Faturamento', 'ID Motivo', 'Situação',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Roteiro', 'ID Usuário',
       'Tabelas', 'Origem', 'Tipo de Entrega', 'Seq Roteiro', 'SKU', 'Seq',
       'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA', 'Total Venda', 'Total AV',
       'Total Geral', 'Margem Bruta R$', 'Comissão R$', 'IPI R$', 'PIS R$',
       'COFINS R$', 'ICMS R$', 'ICMS ST R$', 'Peso Bruto KG',
       'Peso Líquido KG', 'Cad Vendedor','ID Equipe']]

In [58]:
aberto_df=aberto_df.merge(tabelas_df['Vendedor'],on='ID Vendedor',how='inner')[['ID Empresa', 'Pedido', 'ID Cliente', 'ID Vendedor', 'Data do Pedido',
       'SKU', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA', 'Total Venda',
       'Total AV', 'Total Geral', 'Margem Bruta R$', 'Margem Bruta CRP',
       'Situação', 'Peso Bruto KG', 'Peso Líquido KG','ID Equipe']]

In [59]:
tabelas_df['Meta']=tabelas_df['Meta'].merge(tabelas_df['Vendedor'],on='ID Vendedor',how='inner')[['ID Vendedor','Meta R$','ID Equipe']]

In [60]:
tabelas_df['Meta']

,ID Vendedor,Meta R$,ID Equipe
